# Structure Preparation for `CHARMM` or `pyCHARMM` with `crimm`
## In this example, we are going to fetch a structure directly from RCSB by PDB ID. Use crimm to build any missing loops, set the protonation state on the protein residues, and load the structure into CHARMM directly.

### We will use one of the two pdb ids `4pti` or `5wyo` as examples, as is done in the ProteinDynamics folder. This tutorial will illistrate how the `crimm` interface can be used to run `PropKa` to determine the protonation states of ionizable residues and create the necessary patches as well as identifying disulfide bonds and creating the appropriate patches.

### We will use the MMTSB Toolset tool `convpdb.pl` to add solvent and counter ions to each of these two structures.

### Following this, we will illustrate `crimm`'s  loop building capabilities using pdb id `7zap`,  a structure with both protein and RNA present but with missing loop regions. `Crimm` will add the missing loop regions and prepare a complete `CHARMM` psf and pdb for simulation.

### Import basic `crimm` and `pyCHARMM` functionality. Note `pyCHARMM` used through the adaptor is given the prefix `pcm_`.

In [ ]:
from crimm.Fetchers import fetch_rcsb
from crimm.Modeller import ParameterLoader, TopologyLoader
from crimm.Modeller.LoopBuilder import ChainLoopBuilder
from crimm.Modeller.TopoFixer import fix_chain
from crimm.StructEntities import Model
import crimm.Adaptors.pyCHARMMAdaptors as pcm_interface
from crimm.Adaptors.PropKaAdaptors import PropKaProtonator

from pycharmm.psf import delete_atoms as pcm_del_atoms
from pycharmm.psf import get_natom as pcm_get_natom
from pycharmm.generate import patch as pcm_patch
from pycharmm.settings import set_verbosity as pcm_set_verbosity

## Parameters
### Specify first system and the patches to use at the `NTER` and `CTER` positions.
### We'll do 5wyo first since it has two chains.

In [ ]:
pdb_id = '5wyo'
prot_first_patch = 'ACE'
prot_last_patch = 'CT3'
na_first_patch = '5TER'
na_last_patch = '3PHO'
sd_nstep = 300
abnr_nstep = 0
charmm_verbosity_level = 0
pH = 3.5  # Set pH to low value here since we want to test propKa and protonator

## Add some basic functionality to enable `pyCHARMM` minimization in `crimm` and to label special CTER and NTER patching.

In [ ]:
def minimize_chain(chain, sd_nstep, abnr_nstep):
    # load into CHARMM to minimize the structure
    if pcm_get_natom() > 0:
        pcm_del_atoms()
    pcm_interface.load_chain(chain)
    pcm_interface.minimize(sd_nstep=sd_nstep, abnr_nstep=abnr_nstep)
    # Uodate the coordinate in crimm structure
    pcm_interface.sync_coords(chain)

def correct_prot_first_patch(chain, default):
    # PRO and GLY need special treatment when patched at the N-terminus 
    first_resname = chain.residues[0].resname
    if first_resname == 'PRO':
        first_patch = 'PROP'
    elif first_resname == 'GLY':
        first_patch = 'GLYP'
    else:
        first_patch = default
    return first_patch

## This is a `crimm` function constructed to build missing loops in protein structures

In [ ]:
## Separate Chains by Chain Type
### First we need to separate the chain types. Although in this example, we do not have RNA chain, but this routine is built to accommodate both types.

def build_loops(structure,
                prot_first_patch,prot_last_patch,
                na_first_patch,na_last_patch,
                sd_nstep,abnr_nstep,charmm_verbosity_level):
    prot_chains = {}
    na_chains = {}
    # get the first model's id
    model_id = structure.models[0].id
    # create a new empty model to store chains of interests
    new_model = Model(model_id)
    for chain in structure[model_id].chains:
        if chain.chain_type == 'Polypeptide(L)':
            prot_chains[chain.id] = chain
        elif chain.chain_type  in ('Polyribonucleotide', 'Polydeoxyribonucleotide'):
            na_chains[chain.id] = chain

    ## Generate Topology and Loop Building with crimm First
    ### Protein Chains

    for chain_id, chain in prot_chains.items():
        need_minimization = False
        # Missing loop in the chain
        if not chain.is_continuous():
            loop_builder = ChainLoopBuilder(chain)
            # Coordinates of the missing residues will be copied from
            # Alphafold structures
            # only build the loop not the termini
            loop_builder.build_from_alphafold(include_terminal = False)
            chain = loop_builder.get_chain()
            prot_chains[chain_id] = chain
            need_minimization = True
        prot_first_patch = correct_prot_first_patch(chain, default = prot_first_patch)
        rtf_loader['prot'].generate_chain_topology(
            chain,
            first_patch=prot_first_patch, 
            last_patch=prot_last_patch,
            # Coerce any modified residue to canonical residue that it is based on
            coerce=True
        )
        param_loader['prot'].fill_ic(rtf_loader['prot'])
        param_loader['prot'].apply(chain.topo_elements)
        fix_chain(chain)
        if need_minimization:
            # load into CHARMM to minimize the structure
            prev_level = pcm_set_verbosity(charmm_verbosity_level)
            minimize_chain(chain, sd_nstep, abnr_nstep)
            pcm_set_verbosity(prev_level)
        new_model.add(chain)

    ## RNA Chains
    ### DNA chains are not yet supported but will be implemented soon

    for chain_id, chain in na_chains.items():
        # Missing loop is very unlikely in nucleotide chains on PDB
        # but if it exsits, an error will be raise
        if not chain.is_continuous():
            raise ValueError(
                f'Nucleotide chain {chain.id} is not continuous, '
                'topology cannot be generated.'
            )
        rtf_loader['na'].generate_chain_topology(
            chain, 
            first_patch=na_first_patch,
            last_patch=na_last_patch,
            coerce=True
        )
        param_loader['na'].fill_ic(rtf_loader['na'])
        param_loader['na'].apply(chain.topo_elements)
        fix_chain(chain)
        new_model.add(chain)

    ## Finally, replace the model with the new model in the structure

    new_model.set_connect(structure.models[0].connect_dict)
    structure.detach_child(model_id)
    structure.add(new_model)
    return new_model

## Parameter and Topology Loaders
### Choose the relevant `rtf` and `prm` loader for protein and RNA. The respective topology and parameter files will also be streamed into `CHARMM`

In [ ]:
rtf_loader = {
    'prot': TopologyLoader('protein'),
    'na': TopologyLoader('nucleic')
}
param_loader = {
    'prot': ParameterLoader('protein'),
    'na': ParameterLoader('nucleic')
}

# fill the missing ic table values in the respective rtf
for i, (chain_type, cur_rtf) in enumerate(rtf_loader.items()):
    cur_param = param_loader[chain_type]
    cur_param.fill_ic(cur_rtf)
    # load the respective files into CHARMM as well
    prev_level = pcm_set_verbosity(0)
    pcm_interface.load_topology(cur_rtf, append=bool(i))
    pcm_interface.load_parameters(cur_param, append=bool(i))
    pcm_set_verbosity(prev_level)

## Fetch structure from RCSB and visualize it.

In [ ]:
structure = fetch_rcsb(
    pdb_id,
    include_solvent=False,
    # any existing hydrogen will be removed and rebuilt later
    include_hydrogens=False,
    first_model_only=True
)
# Show the structure
structure

### As we can see below in the sequence, this structure has no missing loops (they would be shown in red if there were.

In [ ]:
structure.models[0].chains[0].masked_seq.show()

## The loops are built, and now it is ready for protonation state calculation.

In [ ]:
new_model = build_loops(structure,
                prot_first_patch,prot_last_patch,
                na_first_patch,na_last_patch,
                sd_nstep,abnr_nstep,charmm_verbosity_level)
structure

## Get Protonation State from the specified pH Value
### Note that the protonator accepts Model level entity not the structure itself

In [ ]:
protonator = PropKaProtonator(rtf_loader['prot'], param_loader['prot'], pH = pH)
protonator.load_model(new_model)
protonator.apply_patches()
if pcm_get_natom() > 0:
    pcm_del_atoms()
for chain in new_model:
    if len(protonator.patches[chain.id]) > 0:
        built_atoms = fix_chain(chain)
    # Also load the chain into CHARMM
    pcm_interface.load_chain(chain)

## These protonation patches are identified but not yet loaded into `CHARMM`

In [ ]:
# Construct list of protonated residues to highlight
residues = ''
for i,chainid in enumerate(protonator.patches):
    for j,resid in enumerate(protonator.patches[chainid]):
        if i==0 and j==0: 
            residues += f'{resid}:{chainid} '
        else: 
            residues += f'or {resid}:{chainid} '

## List the computed $pK_A$ values for titratable residues

In [ ]:
df = protonator.to_dataframe()
for i in df.index:
    if i == 0:
        titratable = f'{df.resseq.iloc[i]}:{df.chain_id.iloc[i]}'
    else:
        titratable += f' or {df.resseq.iloc[i]}:{df.chain_id.iloc[i]}'
df

## List of residues being patched

In [ ]:
# Residues being protonated(patched)
import pandas as pd
pd.DataFrame.from_dict(protonator.patches).fillna('-')

## Visualize the structure
> - <h3>Titratable residues are shown as licorice</h3>
> - <h3>Protonation-patched residues are shown as vdW surfaces</h3>

In [ ]:
from crimm.Visualization import View

In [ ]:
view = View()
view.load_entity(new_model)
view.add_representation('licorice',selection=titratable,component=0)
view.add_representation('surface',surfaceType='vws',selection=residues,opacity=0.4,color='lightblue',component=0)
view.add_representation('licorice',selection=titratable,component=1)
view.add_representation('surface',surfaceType='vws',selection=residues,opacity=0.4,color='lightgreen',component=1)
view

## Update `CHARMM` Residues with `pyCHARMM` patch Command

In [ ]:
for chain_id, patch_dict in protonator.patches.items():
    for resid, patch_name in patch_dict.items():
        pcm_patch(patch_name, f'PRO{chain_id} {resid}')

## Check the generated bonds, angle, dihedrals, etc. They should match between `CHARMM` and `crimm`

In [ ]:
new_model.chains[0].topo_elements

## Patch Disulfide Bond
### If any disulfide bond exists in the structure, we will patch them in the `CHARMM` structure. However, disulfide bonds have not been fully implemented in `crimm`.

In [ ]:
if 'disulf' in structure.models[0].connect_dict:
    for res1, res2 in structure.models[0].connect_dict['disulf']:
        seg1, seg2 = res1['chain'], res2['chain']
        seq1, seq2 = res1['resseq'], res2['resseq']
        patch_arg = f'PRO{seg1} {seq1} PRO{seg2} {seq2}'
        print('DISU', patch_arg)
        pcm_patch('DISU', patch_arg)

## Finally, save the structure as PDB and PSF files

In [ ]:
from pycharmm import write
from pycharmm import lingo
import os
lingo.charmm_script('print coor select .not. initialized end')
lingo.charmm_script('hbuild')
lingo.charmm_script('print coor select .not. initialized end')
if not os.path.isdir('pdb'): os.system('mkdir pdb')
write.coor_pdb(f'pdb/{pdb_id}.pdb')
write.psf_card(f'pdb/{pdb_id}.psf')

# Solvate the protein in TIP3P water
## In the following we will use the MMTSB toolset to solvate the blocked alanine residue in a cubic box of TIP3P water using the convpdb.pl commands noted below.

In [ ]:
def get_assigned_segid(file=None):
    # read the pdb to get the assigned segid for each chain
    fpdb = open(file,'r')
    for l in fpdb:
        if l.split()[0] == 'ATOM':
            segid=l.strip().split()[-1]
            break
    fpdb.close()
    return segid

def solvate_and_addions(pdbid=None):
    import numpy as np
    import os
    import pycharmm.psf as psf

    # find the overall charge so we can add neutralizing ions
    q = psf.get_charges()
    Ntot = round((np.sum(q)))
    if Ntot > 0: ion_type = 'CLA'
    if Ntot < 0: ion_type = 'SOD'
    ions = '-ions {}:{}'.format(ion_type,np.abs(Ntot))
    if np.abs(Ntot) < 1e-2: ions = ''
    print(f'Number of ions {np.abs(Ntot)} of type {ion_type}')
    solvate_command = 'convpdb.pl -solvate -cutoff 10 {} -cubic -out charmm22 pdb/{}.pdb > pdb/w.pdb;'\
        .format(ions,pdbid)
    solvate_command +='convpdb.pl -segnames -nsel TIP3 pdb/w.pdb | '
    solvate_command +=f'sed "s/WT0[1,2,3,4,5]/WT00/g" > pdb/{pdbid}_wt00.pdb;'
    solvate_command +='convpdb.pl -segnames -nsel ion pdb/w.pdb > pdb/ions.pdb'
    # run the command as a system subprocess
    os.system(solvate_command)
    # replace HETATM by ATOM in ions
    fpdb = open('pdb/ions.pdb','r')
    opdb = open(f'pdb/{pdbid}_ions.pdb','w')
    for l in fpdb:
        print(l.strip().replace('HETATM','ATOM  '),file=opdb)
    fpdb.close()
    opdb.close()
    # clean-up non-specific files
    os.system('rm pdb/ions.pdb')
    print('Returning')
    return ion_type, ions

In [ ]:
ion_type, ions = solvate_and_addions(pdbid=pdb_id)

In [ ]:
print(ion_type)

In [ ]:
def generate_water_ions(pdbid=None, ions=None):
    '''Generate water segment and minimize the system, protein + solvent + ions, finally save the psf and coordinates
    Note that in "conditioning" the system I first fix the protein atoms and then minimize the water. 
    I am using the steepest descents algorithm because it works best for large systems and/or when you 
    may have bad contacts.

    Input:    pdbid string
              ions string

    Output:   water_segment, ion_segment
    '''
    from pycharmm import read
    from pycharmm import generate as gen
    from pycharmm import settings
    
    read.rtf('../toppar/top_all36_prot.rtf')
    read.prm('../toppar/par_all36m_prot.prm', flex=True)

    read.stream('../toppar/toppar_water_ions.str')
    # 1) **build the water and ion segments**
    water_segment = get_assigned_segid(file=f'pdb/{pdbid}_wt00.pdb')
    # Let's set the wrnlev to 0 to avoid the large output
    old_wrnlev = settings.set_warn_level(0)
    read.sequence_pdb(f'pdb/{pdbid}_wt00.pdb')
    # Now reset back to default wrnlev
    settings.set_warn_level(old_wrnlev)
    # Another example of the generate command
    # generate wt00 noangle nodihedral
    gen.new_segment(water_segment, angle=False, dihedral=False)

    # read coor pdb name pdb/pdbid_wt00.pdb resid
    read.pdb(f'pdb/{pdbid}_wt00.pdb', resid=True)

    # Here is an alternative means of reading a sequence
    # read sequ pdb name pdb/{}_ions.pdb
    # get ion sequence name
    ion_segment = get_assigned_segid(file=f'pdb/{pdbid}_ions.pdb'.format(pdbid))
    read.sequence_pdb(f'pdb/{pdbid}_ions.pdb')

    # Another example of the generate command
    # generate wt00 noangle nodihedral
    gen.new_segment(ion_segment, angle=False, dihedral=False)

    # read coor pdb name pdb/adp.pdb resid
    read.pdb(f'pdb/{pdbid}_ions.pdb', resid=True)

    return (water_segment,ion_segment)

In [ ]:
water_segment, ion_segment = generate_water_ions(pdbid=pdb_id,ions=ions)
print(water_segment,ion_segment)

In [ ]:
def minimize_periodic(pdbid=None):
    '''
    2) **Figure out the boxsize**
    3) **add periodic boundary conditions**
    4) **minimize**
    5) **write psf and final minimized coordinates, ready for simulations**

    Input: pdbid   string
    '''
    import pycharmm
    import pycharmm.energy as energy
    import pycharmm.nbonds as nbonds
    import pycharmm.minimize as minimize
    import pycharmm.crystal as crystal
    import pycharmm.image as image
    import pycharmm.psf as psf
    import pycharmm.read as read
    import pycharmm.write as write
    import pycharmm.cons_fix as cons_fix
    import pycharmm.coor as coor
    import pycharmm.select as select
    
    stats = coor.stat()
    # boxsize
    xsize = stats['xmax'] - stats['xmin']
    ysize = stats['ymax'] - stats['ymin']
    zsize = stats['zmax'] - stats['zmin']
    boxsize = max(xsize, ysize, zsize)

    # half box size
    boxhalf = boxsize / 2.0

    # CHARMM scripting: crystal define cubic @boxsize @boxsize @boxsize 90 90 90
    crystal.define_cubic(boxsize)
    # CHARMM scripting: crystal build cutoff @boxhalf noper 0
    crystal.build(boxhalf)

    # Turn on image centering - bysegment for protein, by residue for solvent and ions
    segids = []
    for segid in psf.get_segid():
        if segid not in [water_segment,ion_segment]:
            segids.append(segid)
            image.setup_segment(0.0, 0.0, 0.0, segid)
    image.setup_residue(0.0, 0.0, 0.0, 'TIP3')
    image.setup_residue(0.0, 0.0, 0.0, ion_type)

    # Now specify nonbonded cutoffs for solvated box
    cutnb = min(boxhalf,12)
    cutim = cutnb
    ctofnb = cutnb - 1.0
    ctonnb = cutnb - 3.0

    # Another nbonds example
    # CHARMM scripting: nbonds cutnb @cutnb cutim @cutim ctofnb @ctofnb ctonnb @ctonnb -
    #        inbfrq -1 imgfrq -1
    pycharmm.NonBondedScript(
        cutnb=cutnb, cutim=cutim, ctonnb=ctonnb, ctofnb=ctofnb,
        eps=1.0,
        cdie=True,
        atom=True, vatom=True,
        fswitch=True, vfswitch=True,
        inbfrq=-1, imgfrq=-1).run()

    # Fix the peptide and minimize the solvent to "fit"
    # CHARMM scripting: cons fix select segid adp end
    cons_fix.setup(pycharmm.SelectAtoms(seg_id=segids))

    # Minimize the solvent positions with periodic boundary conditions using steepest descents
    # CHARMM scripting: mini sd nstep 200 tole 1e-3 tolgrd 1e-3
    minimize.run_sd(nstep=200, tolenr=1e-3, tolgrd=1e-3)

    # Turn off fixed atoms
    # CHARMM scripting: cons fix select none end
    cons_fix.turn_off()

    # Write the psf and coordinates for the solvated peptide
    # write psf card name pdb/adp+wat.psf
    write.psf_card(f'pdb/{pdbid}+wat.psf')
    # write coor pdb name pdb/adp+wat_min.pdb
    write.coor_pdb(f'pdb/{pdbid}+wat_min.pdb')
    return

In [ ]:
minimize_periodic(pdbid=pdb_id)

## Finally, let's visualize the system and see what things look like!

In [ ]:
import nglview as nv
solvated = nv.NGLWidget()
solvated.add_component(f'pdb/{pdb_id}+wat_min.pdb')
solvated.clear_representations()
solvated.add_representation('cartoon',selection='protein', color_scheme='resname')
solvated.add_representation('surface',surfaceType='ms',opacity=0.5,selection='protein', color='lightblue')
solvated.add_representation('spacefill',selection='CLA', color='green')
solvated.add_representation('spacefill',selection='SOD or POT', color='red')
solvated.add_representation('licorice',selection='water',opacity=0.6)
solvated.center()
solvated